# Google Cloud Storage

Cloud Storage是对象存储数据库。

#### 什么是对象？

对象(object)是存储在GCS中的文件，GCS支持任意类型的文件，例如csv,excel,pdf,jpeg,mp4等等。

对象包含两个部分：

* 对象数据：即文件本身包含的数据
* 对象元数据(metadata): 描述对象属性的键值对(key-value pairs)集合

#### 如何命名对象？

GCS存储分区中的对象是相互独立的，本身不存在任何层次关系。

假设创建了两个对象，分别命名为'sh600000.csv'和'kline/daily/sh600001.csv'，尽管后者看起来像处于kline/daily子目录中，但两个文件其实处于同一个命名空间，不存在嵌套关系。

用户仍然可以使用正斜杠分隔符'/'实现"类似"linux文件系统的嵌套目录结构。

In [ ]:
from io import StringIO, BytesIO

import gcsfs
import pandas as pd
from google.cloud import storage

## 如何上传本地文件到GCS?

使用google-cloud-storage实现。

1. 创建Client对象，验证权限，建立连接
2. 创建bucket对象
3. 创建blob对象，调用blob.upload_from_filename

底层实现原理：通过API调用实现。

本地环境使用google-cloud-storage之前，要先设置环境变量'GOOGLE_APPLICATION_CREDENTIALS'

In [ ]:
bucket_name = "financial-data-storage"
blob_name = "csv_file.csv"
local_file = "/home/scofield/gcp/csv_file.csv"  # 务必使用绝对路径

client = storage.Client()
bucket = client.bucket(bucket_name)
blob = bucket.blob(blob_name)
blob.upload_from_filename(local_file)

使用gcsfs库实现。

In [ ]:
# 创建GCS文件系统对象，建立连接
fs = gcsfs.GCSFileSystem("aqueous-cortex-293313")  # 项目ID并不是必须的

# 调用put, put_file方法，前者允许上传多份文件，多线程？异步？
local_file = "/home/scofield/gcp/markdown_file.md"
remote_file = "financial-data-storage/markdown_file.md"  # <bucket_name>/<blob_name>
fs.put_file(lpath=local_file, rpath=remote_file)

## 如何将GCS文件下载到本地？

使用google-cloud-storage实现。

In [ ]:
bucket_name = "financial-data-storage"
blob_name = "image_file.png"
local_file = "/home/scofield/gcp/image_file.png"

client = storage.Client()
bucket = client.bucket(bucket_name)
blob = bucket.blob(blob_name)

# 如果源文件格式和本地文件格式不一致，不报错，但无法打开本地文件，因为无法解码
blob.download_to_filename(local_file)

用gcsfs实现。

In [ ]:
# 创建GCS文件系统对象，建立连接
fs = gcsfs.GCSFileSystem("aqueous-cortex-293313")  # 项目ID并不是必须的

# 调用get, get_file方法
remote_file = "financial-data-storage/markdown_file.md"  # <bucket_name>/<blob_name>
local_file = "/home/scofield/gcp/markdown_file.md"
fs.get_file(rpath=remote_file, lpath=local_file)

## 如何将数据框上传到GCS, 用csv格式存储？

最简单的方法，调用df.to_csv('gcs://bucket_name/blob_name').

从pandas 0.24版本开始，pandas支持与GCS进行数据交互，底层通过gcsfs库实现。

In [ ]:
df = pd.DataFrame({"a": [1, 2, 3], "b": [4, 5, 6]})
df.to_csv("gcs://financial-data-storage/test_df.csv", index=False)

In [ ]:
df = pd.DataFrame({"a": [1, 2, 3], "b": [4, 5, 6]})

bucket_name = "financial-data-storage"
blob_name = "test_df.csv"

client = storage.Client()
bucket = client.bucket(bucket_name)
blob = bucket.blob(blob_name)

csv_file_obj = StringIO()
df.to_csv(csv_file_obj, index=False)
csv_file_obj.seek(0)

blob.upload_from_file(csv_file_obj)

## 如何将GCS的csv加载到数据框？

与上传类似，最简单的方法是调用pd.read_csv('gcs://bucket_name/blob_name')

In [ ]:
df = pd.read_csv("gcs://financial-data-storage/test_df.csv")
df

如何用google-cloud-storage实现？

1. 创建类文件对象file_obj，作为数据交互媒介
2. 创建blob对象，调用blob.download_to_file(file_obj)
3. 调用pd.read_csv(file_obj)，生成数据框

In [ ]:
client = storage.Client()
bucket = client.bucket("financial-data-storage")
blob = bucket.blob("test_df.csv")

# 由于download_to_file下载二进制数据，所以需要使用BytesIO而不是StringIO
file_obj = BytesIO()
blob.download_to_file(file_obj)
file_obj.seek(0)

df = pd.read_csv(file_obj)
df

## 如何将数据框上传到GCS，用parquet存储？

调用df.to_parquet, pd.read_parquet接口。

In [ ]:
df = pd.DataFrame({"a": [1, 2, 3], "b": [4, 5, 6]})
df.to_parquet("gcs://financial-data-storage/test_df.parquet", index=False)

In [ ]:
df = pd.read_parquet("gcs://financial-data-storage/test_df.parquet")
df

如何用google-cloud-storage实现？

In [ ]:
df = pd.DataFrame({"a": [1, 2, 3], "b": [4, 5, 6]})

bucket_name = "financial-data-storage"
blob_name = "test_df.parquet"

client = storage.Client()
bucket = client.bucket(bucket_name)
blob = bucket.blob(blob_name)

file_obj = BytesIO()
df.to_parquet(file_obj, index=False)
file_obj.seek(0)

# content_type默认值是'application/octet-stream'，GCS会根据上传对象的内容和后缀名
# 自动推断并以正确的格式存储
# blob.upload_from_file(file_obj, content_type="application/octet-stream")
blob.upload_from_file(file_obj)